In [2]:
# from jetbot import Robot, Camera
# import torch
# import torchvision.transforms as transforms
# from PIL import Image
# import ipywidgets.widgets as widgets

# robot = Robot()
# camera = Camera.instance(width=224, height=224)


# model = torch.load('my_model.pth')
# model = model.eval().cuda()

# def preprocess(image):
#     image = transforms.functional.to_tensor(image).unsqueeze(0)
#     return image.cuda()

# import os
# os.system("sudo chmod 777/dev/ttyTHS1")

# from servoserial import ServoSerial
# pantilt = ServoSerial()


    

# def motion(class_idx):
#     if class_idx == 0:
#         robot.stop()
#     elif class_idx ==1:
#         robot.forward(0.2)
#     elif class_idx ==2:
#         robot.backward(0.2)
   
        
# while True:
#     image = camera.value
#     output = model(preprocess(image))
#     class_idx = torch.argmax(output).item()
    
#     motion(class_idx)


KeyboardInterrupt: 

In [ ]:
# from jetbot import Robot, Camera
# import torch
# import torchvision.transforms as transforms
# from PIL import Image
# import ipywidgets.widgets as widgets

# robot = Robot()
# camera = Camera.instance(width=224, height=224)


# def predict(image):
#     preprocessed_image = preprocess_image(image)
#     predictions = model.predict(np.expand_dims(preprocessed_image, axis=0))
#     return predictions

# def preprocess_image(image):
#     image = cv2.resize(image, (224, 224))
#     image = image / 255.0 
#     return image

In [ ]:
fgdgdgsgs

In [1]:
import torch
import pickle
model = pickle.load(open("my_model.pth","rb"),encoding="utf-8")
print("모델이 성공적으로 로드되었습니다!")

모델이 성공적으로 로드되었습니다!


In [2]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
robot = Robot()

device = torch.device('cuda')

camera = Camera.instance(width=224, height=224,fps=1)
image = widgets.Image(format='jpeg', width=224, height=224)
stop_slider = widgets.FloatSlider(description='stop', min=0.0, max=1.0, orientation='vertical')
green_slider = widgets.FloatSlider(description='green', min=0.0, max=1.0, orientation='vertical')
orange_slider = widgets.FloatSlider(description='orange', min=0.0, max=1.0, orientation='vertical')
dislike_slider = widgets.FloatSlider(description='dislike', min=0.0, max=1.0, orientation='vertical')


camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([image, dislike_slider,orange_slider,green_slider,stop_slider]))

In [5]:
import cv2
import numpy as np
import torch
import torchvision

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.cuda()
    x = x[None, ...]
    return x

In [6]:
Speed = 0.7
L_F_MPR = 1.00
L_B_MPR =1.00
R_F_MPR= 1.00
R_B_MPR = 1.00

def Move2(speedLeft, speedRight):
    L_F_MPR = 1.00
    L_B_MPR =1.00
    R_F_MPR= 1.00
    R_B_MPR = 1.00
    if speedLeft > 0:
        robot.left_motor.value = speedLeft * L_F_MPR
    else:
        robot.left_motor.value = speedLeft * L_B_MPR
    if speedRight > 0:
        robot.right_motor.value = speedRight * R_F_MPR
    else:
        robot.right_motor.value = speedRight * R_B_MPR

def Move(speed):
    Move2(speed, speed)

def Stop():
    robot.left_motor.value = 0
    robot.right_motor.value = 0


In [13]:
import torch.nn.functional as F
import time


def update(change):
    global robot
    x = change['new']
    x = preprocess(x)
    print(x)
    y = model((x))
    print(y)
    
    y = F.softmax(y,dim=1)
    
    prob_stop = float(y.flatten()[3])
    prob_orange = float(y.flatten()[1])
    prob_green = float(y.flatten()[0])
    prob_dislike = float(y.flatten()[2])
    
    stop_slider.value = prob_stop
    green_slider.value = prob_green
    orange_slider.value = prob_orange 
    dislike_slider.value = prob_dislike 
    
    if prob_stop > 0.6:
        robot.stop()
        
    elif prob_green > 0.6:      
        robot.forward(0.8)
        time.sleep(0.5)
        robot.stop()
        
    elif prob_orange >0.6:  
        robot.backward(0.8)
        time.sleep(0.5)
        robot.stop()
        
update({'new': camera.value})
camera.observe(update,names = 'value')  

tensor([[[[-0.0801,  0.1083, -0.0801,  ...,  1.2899,  1.7694, -0.2342],
          [-0.1143, -0.0801,  0.0912,  ..., -0.8507,  0.0398, -0.2342],
          [ 0.0056,  0.0056,  0.0398,  ..., -0.9705, -0.8507, -0.3369],
          ...,
          [-0.0629,  0.1254,  0.0056,  ...,  0.0227, -0.0458, -0.0972],
          [ 0.0569,  0.2111, -0.1314,  ..., -0.0287, -0.1657, -0.2856],
          [-0.0629, -0.0629, -0.1486,  ..., -0.2684, -0.3369, -0.1999]],

         [[ 0.6604,  0.6429,  0.7304,  ...,  1.6232,  2.3060,  0.2052],
          [ 0.7479,  0.7304,  0.7654,  ..., -0.3550,  0.6954,  0.2577],
          [ 0.7479,  0.7129,  0.6954,  ..., -0.7752, -0.5476,  0.2227],
          ...,
          [ 0.8704,  1.0980,  0.9930,  ..., -0.0574, -0.1975, -0.0574],
          [ 1.0280,  1.0280,  1.0105,  ..., -0.0574, -0.2150, -0.2500],
          [ 0.8704,  0.9230,  0.9405,  ..., -0.4426, -0.3375, -0.3901]],

         [[ 1.4548,  1.4025,  1.4548,  ...,  2.3263,  2.6400,  1.0017],
          [ 1.4548,  1.4548,  

KeyboardInterrupt: 

In [ ]:
camera.observe(update, names = 'value')

In [7]:
update({'new': camera.value})
camera.observe(update,names = 'value')

119547037146038801333356

In [9]:
    elif prob_dislike > 0.6:

SyntaxError: invalid syntax (<ipython-input-9-176aba890146>, line 1)

In [ ]:
elif prob_dislike >0.6:
        for i in range(0,repeat=3):
            pantilt.Servo_serial_control(1, MID_PAN+200)
            Move2(-speed, speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN+400)
            Move2(-speed, speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN+200)
            Stop()
            time.sleep(delay)

            pantilt.Servo_serial_control(1, MID_PAN-200)
            Move2(speed, -speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN-400)
            Move2(speed, -speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN-200)
            Stop()
            time.sleep(delay)

            pantilt.Servo_serial_control(1, MID_PAN)
            Stop()
            time.sleep(delay)

In [3]:
import torch
import pickle
import cv2
from jetbot import Robot, Camera
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg


# Load the pretrained model
model = pickle.load(open("my_model.pth", "rb"), encoding="utf-8")
print("모델이 성공적으로 로드되었습니다!")

# Initialize the robot and camera
robot = Robot()
camera = Camera.instance(width=224, height=224, fps=1)

# Create widgets for displaying images and sliders
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

# Function to preprocess image for the model
def preprocess(image):
    # Your image preprocessing code here (if needed)
    return image

# Function to update robot actions based on color detection
def update(change):
    image_value = change['new']
    # Preprocess the image
    preprocessed_image = preprocess(image_value)
    
    # Make predictions using the model
    predictions = model(preprocessed_image)
    
    # Get the predicted class (index with the maximum probability)
    predicted_class = torch.argmax(predictions).item()
    
    # Display the image with detected color
    image.value = bgr8_to_jpeg(image_value)
    
    # Define color thresholds (adjust as needed)
    red_threshold = 150
    green_threshold = 150
    blue_threshold = 150
    
    # Extract color values from the center of the image
    center_pixel = image_value[112, 112, :]
    red_value, green_value, blue_value = center_pixel
    
    # Control the robot based on the detected color
    if red_value > red_threshold:
        # Stop for red
        robot.stop()
    elif green_value > green_threshold:
        # Move forward for green
        robot.forward(0.5)
    elif blue_value > blue_threshold:
        # Move backward for blue
        robot.backward(0.5)

# Observe the camera value and update the robot actions
camera.observe(update, names='value')


모델이 성공적으로 로드되었습니다!


RuntimeError: Could not initialize camera.  Please see error trace.

In [2]:
import os
os.system("sudo chmod 777 /dev/ttyTHS1")

from servoserial import ServoSerial

ModuleNotFoundError: No module named 'servoserial'